<div style="display: flex; background-color: RGB(255,114,0);" >

# PROJET - Logement </h1>
</div>

> Construire un modèle de régression à l'aide de l'API/ Mlflow

Pour mettre en œuvre des réseaux de neurones en utilisant Keras API. Il s'agit d'une API de haut niveau magnifiquement conçue et simple pour la construction, former, évaluer et faire fonctionner des réseaux de neurones.

Pour s'initier à l'API,  vous  allez entraîner un modèle de régression afin de prédire le prix médian des maisons de la Californie. Ensuite, vous déploierez ce modèle à travers une API REST en utilisant la librairie  MLflow.

<div style="display: flex; background-color: blue;" >

# 1.Contexte du projet
</div>

C’est un projet sur le logement en Californie dans le but de prédire le prix médian des maisons de la Californie. Pour plus de simplicité, vous allez utiliser la fonction fetch_california_housing() de Scikit-Learn pour charger les données. Cet ensemble de données est plus simple que celui que nous avons utilisé auparavant, puisqu'il ne contient que des caractéristiques numériques (il n'y a pas la variable ocean_proximity), et il y a pas des valeurs manquantes . (Le choix du modèle n'est pas très important pour ce projet, mais essayez d’aborder le sujet à l'aide d’un modèle ensembliste)

- Entraîner un modèle de régression afin de prédire le prix médian des maisons de la Californie.
- Ensuite, appliquer ce modèle à travers une API REST en utilisant la librairie Mlflow.
- Finalement, tester vos prédictions avec une interface Streamlit.

## Ressources :
- https://openclassrooms.com/fr/courses/4525361-realisez-un-dashboard-avec-vos-donnees/5774786-apprehendez-le-fonctionnement-dun-serveur-web
- https://openclassrooms.com/fr/courses/6573181-adoptez-les-api-rest-pour-vos-projets-web
- https://practicalprogramming.fr/api-rest/
- https://www.mlflow.org/docs/latest/index.html
- https://docs.microsoft.com/fr-fr/azure/databricks/applications/mlflow/
- https://streamlit.io/
- https://docs.microsoft.com/fr-fr/azure/databricks/_static/notebooks/mlflow/mlflow-quick-start-deployment-azure.html

<div style="display: flex; background-color: green;" >

## 1.2. Util
</div>

Créer une API Python :
- ``Django`` : Django est basé sur Python. Il est utilisé par de grands noms comme Google, YouTube et Instagram. Le framework REST de Django est facile à utiliser lorsque vous construisez vos API REST avec Django. Il demande un effort d’apprentissage aux débutants, mais possède d’excellentes fonctionnalités intégrées, comme l’authentification et la messagerie. - https://openclassrooms.com/fr/courses/7192416-mettez-en-place-une-api-avec-django-rest-framework
- ``Flask`` : Flask utilise Python pour le web et le développement des API REST. C’est un framework minimaliste, facile d’apprentissage et d’utilisation. Flask comprend moins de fonctionnalités intégrées que Django, mais permet aux développeurs d’avoir davantage de choix dans les outils additionnels qu’ils utilisent.

In [1]:
from os import getcwd
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import Pipeline
import datetime

In [2]:
verbose = 0

<div style="display: flex; background-color: blue;" >

# 2.Chargement des données
</div>

In [3]:
X, y = fetch_california_housing(return_X_y=True)

In [4]:
print(X.shape, y.shape)

(20640, 8) (20640,)


<div style="display: flex; background-color: indigo;" >

#### Split train / test
</div>

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
random_state=0

In [7]:
# split 80/20 train-test
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=random_state)

<div style="display: flex; background-color: indigo;" >

#### Standardisation
</div>

In [8]:
scaler = preprocessing.StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

<div style="display: flex; background-color: indigo;" >

#### Recherche du meilleur modèle
</div>

In [9]:
models_list = { }

In [10]:
grid_params = { 
                'learning_rate' : [1e-1, 2e-1, 3e-1],
                'max_depth':[2,4,5],
                'min_samples_leaf':[30,31,32],
                'max_iter':[100,150],
                'verbose':[verbose], 
                'random_state':[random_state]
                }

model = HistGradientBoostingRegressor()
grid = GridSearchCV(model,param_grid=grid_params, cv=4)
models_list["HistGradientBoostingRegressor"] = grid

In [11]:
from api_logement_function import fit_and_test_models

In [12]:
modeldic, scores = fit_and_test_models(models_list, X_train_scaled, y_train, X_test_scaled, y_test, y_column_name=None, scores=None, verbose=verbose, metrics=1)

score_all_class_df = pd.DataFrame(scores).set_index("Modeli")
score_all_class_df = score_all_class_df.round(decimals=3)
score_all_class_df

,Class,Model,R2,fit time,fit seconde,metrics time,metrics seconde,predict time,predict seconde,MAE,MSE,RMSE,Mediane AE
Modeli,,,,,,,,,,,,,
HistGradientBoostingRegressor,,HistGradientBoostingRegressor,0.843,00:01:54,114.066,00:00:00,0.025,00:00:00,0.025,0.306,0.205,0.453,0.205


In [13]:
grid = modeldic["HistGradientBoostingRegressor"]
print(grid.best_score_)
print(grid.best_params_)

0.8291532722535846
{'learning_rate': 0.2, 'max_depth': 5, 'max_iter': 150, 'min_samples_leaf': 32, 'random_state': 0, 'verbose': 0}


<div style="display: flex; background-color: indigo;" >

#### Création du pipeline avec le meilleur modèle
</div>

In [14]:
pipeline = Pipeline([
        ('scaler', preprocessing.StandardScaler()),
        ('mlfow_model', HistGradientBoostingRegressor(**grid.best_params_))
        ])

In [15]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('mlfow_model',
                 HistGradientBoostingRegressor(learning_rate=0.2, max_depth=5,
                                               max_iter=150,
                                               min_samples_leaf=32,
                                               random_state=0))])

In [16]:
pipeline.score(X_test, y_test)

0.8429720051432265

<div style="display: flex; background-color: indigo;" >

#### Sauvegarde du modèle entrainé
</div>

In [17]:
from joblib import dump, load
from datetime import datetime

In [18]:
def save_model(model_to_save, file_path, model_save_file_name):
    # Sauvegarde du meilleur modele
    now = datetime.now() # current date and time
    date_time = now.strftime("%Y-%m-%d-%H_%M_%S")
    model_save_file_name = model_save_file_name + date_time + '.joblib'
    # Attention, il faudra mettre à jour les colonnes correspondantes dans le premier if en cas de modification du model
    dump(model_to_save, file_path+model_save_file_name)
    return file_path+model_save_file_name

In [19]:
model_path = save_model(pipeline, getcwd()+"\\", model_save_file_name='HistGradientBoostingRegressor_')
model_path

'c:\\Users\\User\\WORK\\workspace-ia\\PROJETS\\projet_api_logement_aurelie\\projet_api_logement_aurelie_mlflow_simple\\HistGradientBoostingRegressor_2022-05-19-10_14_23.joblib'

<div style="display: flex; background-color: indigo;" >

#### Création de la signature
</div>

In [20]:
from mlflow.models.signature import infer_signature

In [21]:
signature = infer_signature(X_train, y_train)

<div style="display: flex; background-color: indigo;" >

#### Environnement
</div>

In [22]:
from sys import version_info

In [23]:
PYTHON_VERSION = "{major}.{minor}.{micro}".format(major=version_info.major,
                                              minor=version_info.minor,
                                              micro=version_info.micro)
PYTHON_VERSION

'3.9.10'

<div style="display: flex; background-color: indigo;" >

#### Ajout du modèle dans ML Flow
</div>

In [24]:
import mlflow.sklearn

In [25]:
nom_model = 'mlfow_model'

In [26]:
mlflow.sklearn.save_model(pipeline, nom_model, signature=signature)

C:\Program Files\Python39\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Commande à lancer dans un terminal

In [29]:
print(f"mlflow models serve -m {nom_model}/")
print(f"mlflow models serve -m {nom_model}/ --no-conda")
print(f"mlflow models serve -m {nom_model}/ --env-manager=local")

mlflow models serve -m mlfow_model/
mlflow models serve -m mlfow_model/ --no-conda
mlflow models serve -m mlfow_model/ --env-manager=local
